In [3]:
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
dataset_path = "C:/Users/aniks/OneDrive/Documents/a/dataset"

# Define parameters
img_size = (128, 128)
batch_size = 32

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    preprocessing_function=lambda x: cv2.cvtColor(x, cv2.COLOR_BGRA2BGR) if x.shape[2] == 4 else x
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Build the model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Save the model
model.save('mask_detection_model.h5')


Found 818 images belonging to 2 classes.
Epoch 1/10
103/103 [==============================] - 46s 440ms/step - loss: 0.4288 - accuracy: 0.8180 - val_loss: 0.2516 - val_accuracy: 0.9303
Epoch 2/10
103/103 [==============================] - 45s 441ms/step - loss: 0.2359 - accuracy: 0.9108 - val_loss: 0.1268 - val_accuracy: 0.9633
Epoch 3/10
103/103 [==============================] - 46s 443ms/step - loss: 0.1819 - accuracy: 0.9313 - val_loss: 0.1296 - val_accuracy: 0.9645
Epoch 4/10
103/103 [==============================] - 46s 450ms/step - loss: 0.1518 - accuracy: 0.9481 - val_loss: 0.1054 - val_accuracy: 0.9670
Epoch 5/10
103/103 [==============================] - 45s 439ms/step - loss: 0.1343 - accuracy: 0.9511 - val_loss: 0.1227 - val_accuracy: 0.9645
Epoch 6/10
103/103 [==============================] - 47s 452ms/step - loss: 0.1031 - accuracy: 0.9621 - val_loss: 0.1215 - val_accuracy: 0.9572
Epoch 7/10
103/103 [==============================] - 47s 459ms/step - loss: 0.0832 - acc

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('mask_detection_model.h5')

# Initialize webcam
cap = cv2.VideoCapture(0)

# Define the labels for mask and no mask
labels = {0: 'Without Mask', 1: 'With Mask'}

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Preprocess the frame
    resized_frame = cv2.resize(frame, img_size)
    normalized_frame = resized_frame / 255.0
    reshaped_frame = np.reshape(normalized_frame, (1, *img_size, 3))

    # Make predictions
    predictions = model.predict(reshaped_frame)
    label = labels[int(predictions[0, 0] > 0.5)]

    # Display the result on the frame
    cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Mask Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 32ms/step
